<a href="https://colab.research.google.com/github/aarchi-dot/deep-learning/blob/main/lab3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ===== VGG + BCE + Adam (MNIST) =====
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Data
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.Grayscale(3),
    transforms.ToTensor()
])

train_ds = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_ds  = datasets.MNIST(root='./data', train=False, download=True, transform=transform)

train_loader = DataLoader(train_ds, batch_size=64, shuffle=True)
test_loader  = DataLoader(test_ds, batch_size=64)

# Model
model = models.vgg16(pretrained=True)
model.classifier[6] = nn.Linear(4096, 10)
model.to(device)

# BCE Loss
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

# Training
for epoch in range(10):
    model.train()
    correct = total = 0
    for x, y in train_loader:
        x, y = x.to(device), y.to(device)
        y_onehot = torch.zeros(y.size(0), 10).to(device)
        y_onehot.scatter_(1, y.unsqueeze(1), 1)

        optimizer.zero_grad()
        out = model(x)
        loss = criterion(out, y_onehot)
        loss.backward()
        optimizer.step()

        pred = out.argmax(1)
        correct += pred.eq(y).sum().item()
        total += y.size(0)

    print(f"Epoch {epoch+1} | Train Acc: {100*correct/total:.2f}%")

# Testing
model.eval()
correct = total = 0
with torch.no_grad():
    for x, y in test_loader:
        x, y = x.to(device), y.to(device)
        out = model(x)
        pred = out.argmax(1)
        correct += pred.eq(y).sum().item()
        total += y.size(0)

print(f"Test Accuracy: {100*correct/total:.2f}%")
